<a href="https://colab.research.google.com/github/itz-sidd/2D-to-3D-Image-Converter/blob/main/image_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!git clone https://github.com/pyimagesearch/TripoSR.git
import sys
sys.path.append('/content/TripoSR/tsr')
%cd TripoSR
!pip install -r requirements.txt -q

Cloning into 'TripoSR'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 164 (delta 65), reused 41 (delta 41), pack-reused 67 (from 1)
Receiving objects: 100% (164/164), 36.71 MiB | 24.11 MiB/s, done.
Resolving deltas: 100% (67/67), done.
/content/TripoSR
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.5/688.5 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.9/227.9 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
!pip install onnxruntime


In [9]:
import torch
import os
import time
from PIL import Image
import numpy as np
from IPython.display import Video
from tsr.system import TSR
from tsr.utils import remove_background, resize_foreground, save_video
import pymeshlab as pymesh
import rembg

device = "cuda" if torch.cuda.is_available() else "cpu"

class Timer:
    def __init__(self):
        self.items = {}
        self.time_scale = 1000.0  # ms
        self.time_unit = "ms"
    def start(self, name: str) -> None:
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        self.items[name] = time.time()
    def end(self, name: str) -> float:
        if name not in self.items:
            return
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        start_time = self.items.pop(name)
        delta = time.time() - start_time
        t = delta * self.time_scale
        print(f"{name} finished in {t:.2f}{self.time_unit}.")
timer = Timer()

In [25]:
from google.colab import files
uploaded = files.upload()
original_image = Image.open(list(uploaded.keys())[0])
original_image.resize((512, 512)).save("examples/product.png")

Saving products.png to products (1).png


In [11]:
!pip install --upgrade --force-reinstall Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.3 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 10.1.0
    Uninstalling Pillow-10.1.0:
      Successfully uninstalled Pillow-10.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.


In [26]:
import os
import time  # <-- ADD THIS

# --- ADD THIS ENTIRE TIMER CLASS ---
class Timer:
    def __init__(self):
        self.items = {}
        self.time_scale = 1000.0  # ms

    def start(self, name: str) -> None:
        self.items[name] = time.time()

    def end(self, name: str) -> float:
        if name not in self.items:
            return 0.0
        start_time = self.items[name]
        end_time = time.time()
        self.items[name] = (end_time - start_time) * self.time_scale
        return self.items[name]
# --- END OF TIMER CLASS ---

# --- Your existing code ---
image_paths = "/content/TripoSR/examples/product.png"
device = "cuda:0"
pretrained_model_name_or_path = "stabilityai/TripoSR"
chunk_size = 8192
no_remove_bg = True
foreground_ratio = 0.85
output_dir = "output/"
model_save_format = "obj"
render = True
output_dir = output_dir.strip()
os.makedirs(output_dir, exist_ok=True)

timer = Timer()  # <-- CREATE AN INSTANCE
timer.start("Initializing model")
model = TSR.from_pretrained(
    pretrained_model_name_or_path,
    config_name="config.yaml",
    weight_name="model.ckpt",
)
model.renderer.set_chunk_size(chunk_size)
model.to(device)
timer.end("Initializing model")

6525.899410247803

In [27]:
timer.start("Processing images")
images = []
rembg_session = rembg.new_session()
image = remove_background(original_image, rembg_session)
image = resize_foreground(image, foreground_ratio)
if image.mode == "RGBA":
    image = np.array(image).astype(np.float32) / 255.0
    image = image[:, :, :3] * image[:, :, 3:4] + (1 - image[:, :, 3:4]) * 0.5
    image = Image.fromarray((image * 255.0).astype(np.uint8))
image_dir = os.path.join(output_dir, str(0))
os.makedirs(image_dir, exist_ok=True)
image.save(os.path.join(image_dir, "input.png"))
images.append(image)
timer.end("Processing images")

2427.354097366333

In [28]:
for i, image in enumerate(images):
    print(f"Running image {i + 1}/{len(images)} ...")
    timer.start("Running model")
    with torch.no_grad():
        scene_codes = model([image], device=device)
    timer.end("Running model")

    if render:
        timer.start("Rendering")
        render_images = model.render(scene_codes, n_views=30, return_type="pil")
        for ri, render_image in enumerate(render_images[0]):
            render_image.save(os.path.join(output_dir, str(i), f"render_{ri:03d}.png"))
        save_video(
            render_images[0], os.path.join(output_dir, str(i), "render.mp4"), fps=30
        )
        timer.end("Rendering")

    timer.start("Exporting mesh")
    meshes = model.extract_mesh(scene_codes, has_vertex_color=False)
    mesh_file = os.path.join(output_dir, str(i), f"mesh.{model_save_format}")
    meshes[0].export(mesh_file)
    timer.end("Exporting mesh")
print("Processing complete.")

Running image 1/1 ...
Processing complete.


In [29]:
obj_file = "/content/TripoSR/output/0/mesh.obj"
# Load the .obj mesh
ms = pymesh.MeshSet()
ms.load_new_mesh(obj_file)
mesh = ms.current_mesh()
# Convert to .stl format
stl_file = 'model.stl'
ms.save_current_mesh(stl_file)

In [30]:
Video('output/0/render.mp4', embed=True)